In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../data/vitals_time.csv')

In [ ]:
df = df.rename(columns={'Unnamed: 0': 'Timestamp'})
df.set_index('Timestamp',inplace=True)

In [ ]:
df=df.dropna()

In [ ]:
df.isna().sum()

heartrate    0
resprate     0
o2sat        0
sbp          0
dbp          0
dtype: int64

In [ ]:
df

,heartrate,resprate,o2sat,sbp,dbp
Timestamp,,,,,
2023-05-06 00:00:00,83.0,24.0,97.0,90.0,51.0
2023-05-06 00:00:01,85.0,22.0,98.0,76.0,39.0
2023-05-06 00:00:02,84.0,22.0,97.0,75.0,39.0
2023-05-06 00:00:03,84.0,20.0,99.0,86.0,51.0
2023-05-06 00:00:04,86.0,20.0,98.0,65.0,37.0
...,...,...,...,...,...
2023-05-07 06:42:37,69.0,22.0,100.0,127.0,53.0
2023-05-07 06:42:38,67.0,16.0,99.0,140.0,74.0
2023-05-07 06:42:39,65.0,16.0,99.0,125.0,79.0


In [ ]:
data=df
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110562 entries, 2023-05-06 00:00:00 to 2023-05-07 06:42:41
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   heartrate  110562 non-null  float64
 1   resprate   110562 non-null  float64
 2   o2sat      110562 non-null  float64
 3   sbp        110562 non-null  float64
 4   dbp        110562 non-null  float64
dtypes: float64(5)
memory usage: 5.1+ MB


In [ ]:
train_df = df.iloc[:-1000]
val_df = df.iloc[-1000:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_df)
val_data = scaler.transform(val_df)


In [ ]:
BATCH_SIZE = 256
train_ds = tf.data.Dataset.from_tensor_slices((train_data[:-1], train_data[1:])).batch(BATCH_SIZE).shuffle(len(train_data))
val_ds = tf.data.Dataset.from_tensor_slices((val_data[:-1], val_data[1:])).batch(BATCH_SIZE)

In [ ]:
inputs = tf.keras.layers.Input(shape=(train_data.shape[1],))
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(32, activation='relu')(x)
outputs = tf.keras.layers.Dense(train_data.shape[1], activation='linear')(x)
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
EPOCHS = 100
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Epoch 1/100
428/428 [==============================] - 3s 3ms/step - loss: 0.0014 - val_loss: 6.2580e-04
Epoch 2/100
428/428 [==============================] - 1s 2ms/step - loss: 7.1113e-04 - val_loss: 6.1639e-04
Epoch 3/100
428/428 [==============================] - 1s 2ms/step - loss: 7.0758e-04 - val_loss: 6.1190e-04
Epoch 4/100
428/428 [==============================] - 1s 2ms/step - loss: 7.0643e-04 - val_loss: 6.1643e-04
Epoch 5/100
428/428 [==============================] - 1s 2ms/step - loss: 7.0535e-04 - val_loss: 6.0882e-04
Epoch 6/100
428/428 [==============================] - 1s 2ms/step - loss: 7.0590e-04 - val_loss: 6.1609e-04
Epoch 7/100
428/428 [==============================] - 1s 2ms/step - loss: 7.0356e-04 - val_loss: 6.0916e-04
Epoch 8/100
428/428 [==============================] - 1s 2ms/step - loss: 7.0304e-04 - val_loss: 6.1798e-04
Epoch 9/100
428/428 [==============================] - 1s 2ms/step - loss: 7.0667e-04 - val_loss: 6.0778e-04
Epoch 10/100
428/428 [=

In [ ]:
#last_row=train_data[-1:]

In [ ]:
#last_row=train_data.iloc[[i]]

In [ ]:
#(last_row)

In [ ]:
#last_rows = train_data.iloc[max(0, 1-4):1+1]

In [ ]:
#last_rows = train_data.iloc[1:1+5]
#last_row = last_rows.iloc[:,:-1]

In [ ]:
'''next_values = []
for i in range(5):
    # Predict the next set of values
    predicted_values_1 = model.predict(last_row)
    #print(predicted_values_1)
    # Inverse transform the predicted values using the scaler
    predicted_values = scaler.inverse_transform(predicted_values_1)
    print(predicted_values)
    # Add the predicted values to the list
    next_values.append(predicted_values)

    # Update the last row with the predicted values
    last_row =  np.vstack([last_row[1:], predicted_values])

# Print the predicted values
next_values = np.array(next_values).reshape((5, train_data.shape[1]))
#print(next_values)'''

'next_values = []\nfor i in range(5):\n    # Predict the next set of values\n    predicted_values_1 = model.predict(last_row)\n    #print(predicted_values_1)\n    # Inverse transform the predicted values using the scaler\n    predicted_values = scaler.inverse_transform(predicted_values_1)\n    print(predicted_values)\n    # Add the predicted values to the list\n    next_values.append(predicted_values)\n\n    # Update the last row with the predicted values\n    last_row =  np.vstack([last_row[1:], predicted_values])\n\n# Print the predicted values\nnext_values = np.array(next_values).reshape((5, train_data.shape[1]))\n#print(next_values)'

In [ ]:
import random as rand

In [ ]:
rand.seed(109)
sbp_L, sbp_H = 40, 150
dbp_L, dbp_H = 30, 120
hr_L, hr_H = 20, 250
o2sat_L, o2sat_H = 30, 190
rr_L, rr_H = 5, 35

In [ ]:
'''#n=0
#without instabillty prediction
next_values = []
data_sim = pd.DataFrame(columns=['o2sat', 'sbp', 'dbp', 'heartrate', 'resprate'])
while True:
  #last_row=[]
  sbp = [rand.randint(sbp_L,sbp_H)]
  dbp = [rand.randint(dbp_L,dbp_H)]
  heartrate = [rand.randint(hr_L,hr_H)]
  o2sat = [rand.randint(o2sat_L,o2sat_H)]
  resprate = [rand.randint(rr_L,rr_H)]
  data={'o2sat':o2sat, 'sbp':sbp, 'dbp':dbp, 'heartrate':heartrate, 'resprate':resprate}
  df2 = pd.DataFrame(data)
  data_sim = pd.concat([data_sim, df2], ignore_index=True)
  print(data_sim)
  last_row=data_sim[-1:].to_numpy().astype('float32')
  last_row=scaler.fit_transform(last_row)
  for i in range(5):
    # Predict the next set of values
    predicted_values_1 = model.predict(last_row)
    #print(predicted_values_1)
    # Inverse transform the predicted values using the scaler
    predicted_values = scaler.inverse_transform(predicted_values_1)
    print(predicted_values)
    # Add the predicted values to the list
    next_values.append(predicted_values)
    # Update the last row with the predicted values
    #last_row =  np.vstack([last_row[1:], np.array(next_values[-5:]).reshape((5, last_row.shape[1]))])
    #last_row =  np.vstack([last_row[1:], np.array(next_values[-5:]).reshape((5, last_row.shape[1]))])
    last_row = np.vstack([last_row[1:], predicted_values])
  # next_values = np.array(next_values).reshape((5, 1))
  # next_values = np.array(next_values).reshape((5, 1))
    tf.keras.backend.clear_session()
 # next_values = np.array(next_values).reshape((2, 1))'''

"#n=0\n#without instabillty prediction\nnext_values = []\ndata_sim = pd.DataFrame(columns=['o2sat', 'sbp', 'dbp', 'heartrate', 'resprate'])\nwhile True:\n  #last_row=[]\n  sbp = [rand.randint(sbp_L,sbp_H)]\n  dbp = [rand.randint(dbp_L,dbp_H)]\n  heartrate = [rand.randint(hr_L,hr_H)]\n  o2sat = [rand.randint(o2sat_L,o2sat_H)]\n  resprate = [rand.randint(rr_L,rr_H)]\n  data={'o2sat':o2sat, 'sbp':sbp, 'dbp':dbp, 'heartrate':heartrate, 'resprate':resprate}\n  df2 = pd.DataFrame(data)\n  data_sim = pd.concat([data_sim, df2], ignore_index=True)\n  print(data_sim)\n  last_row=data_sim[-1:].to_numpy().astype('float32')\n  last_row=scaler.fit_transform(last_row)\n  for i in range(5):\n    # Predict the next set of values\n    predicted_values_1 = model.predict(last_row)\n    #print(predicted_values_1)\n    # Inverse transform the predicted values using the scaler\n    predicted_values = scaler.inverse_transform(predicted_values_1)\n    print(predicted_values)\n    # Add the predicted values to 

In [ ]:
sbp_threshold = 90
hr_threshold = 100
o2sat_threshold = 90
rr_threshold = 20
#instability = ((df['sbp'] < sbp_threshold) & (df['heartrate'] > hr_threshold)) | \
              #((df['o2sat'] < o2sat_threshold) & (df['resprate'] > rr_threshold))

In [ ]:
rolling_mean_data = df.rolling(5).mean().dropna()

In [ ]:
rolling_mean_data['instability'] = np.where(((rolling_mean_data['sbp'] < sbp_threshold) & (rolling_mean_data['heartrate'] > hr_threshold)) | \
              ((rolling_mean_data['o2sat'] < o2sat_threshold) & (rolling_mean_data['resprate'] > rr_threshold)), 1, 0)

In [ ]:
rolling_mean_data

,heartrate,resprate,o2sat,sbp,dbp,instability
Timestamp,,,,,,
2023-05-06 00:00:04,84.4,21.6,97.8,78.4,43.4,0
2023-05-06 00:00:05,84.8,20.0,98.2,77.0,42.2,0
2023-05-06 00:00:06,84.8,19.2,98.2,78.0,42.0,0
2023-05-06 00:00:07,83.8,18.0,98.4,84.4,46.2,0
2023-05-06 00:00:08,86.2,17.4,98.6,87.6,47.6,0
...,...,...,...,...,...,...
2023-05-07 06:42:37,73.2,17.0,99.0,125.6,67.2,0
2023-05-07 06:42:38,73.0,17.0,99.4,131.8,70.6,0
2023-05-07 06:42:39,69.6,17.0,99.2,129.2,71.4,0


In [ ]:
train_size = int(len(rolling_mean_data) * 0.85)
train_data_roll, test_data_roll = rolling_mean_data[:train_size], rolling_mean_data[train_size:]

In [ ]:

X_train = scaler.fit_transform(train_data_roll[['o2sat', 'sbp', 'dbp', 'heartrate', 'resprate']])
y_train = train_data_roll['instability']
X_test = scaler.transform(test_data_roll[['o2sat', 'sbp', 'dbp', 'heartrate', 'resprate']])
y_test = test_data_roll['instability']

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

GradientBoostingClassifier()

In [ ]:
#df[:1]

In [ ]:
'''#n=0
next_values = []
data_sim = pd.DataFrame(columns=['o2sat', 'sbp', 'dbp', 'heartrate', 'resprate'])
#next_df= pd.DataFrame(columns=['o2sat', 'sbp', 'dbp', 'heartrate', 'resprate'])
while True:
  #last_row=[]
  
  print(data_sim)
  last_row=df[-1:].to_numpy().astype('float32')
  last_row=scaler.fit_transform(last_row)
  for i in range(5):
    # Predict the next set of values
    predicted_values_1 = model.predict(last_row)
    #print(predicted_values_1)
    # Inverse transform the predicted values using the scaler
    predicted_values = scaler.inverse_transform(predicted_values_1)
    print(predicted_values)
    # Add the predicted values to the list
    next_values.append(predicted_values)
    # Update the last row with the predicted values
    #last_row =  np.vstack([last_row[1:], np.array(next_values[-5:]).reshape((5, last_row.shape[1]))])
    #last_row =  np.vstack([last_row[1:], np.array(next_values[-5:]).reshape((5, last_row.shape[1]))])
    last_row = np.vstack([last_row[1:], predicted_values])
    # Calculate the rolling average of the predicted values for the next 5 rows
  rolling_avg = np.mean(np.array(next_values), axis=0)
  #print("Rolling average of the predicted values:", rolling_avg)
  rolling_df_with_is=pd.DataFrame(rolling_avg,columns=['o2sat', 'sbp', 'dbp', 'heartrate', 'resprate'])
  
  X = scaler.transform(rolling_df_with_is[['o2sat', 'sbp', 'dbp', 'heartrate', 'resprate']])
  y_pred = gb.predict(X)
  rolling_df_with_is['predicted_instability'] = y_pred
  print("Rolling average of the predicted values:",rolling_df_with_is )
  if y_pred[0] == 1:
    print("Instability predicted")
  # next_values = np.array(next_values).reshape((5, 1))
  # next_values = np.array(next_values).reshape((5, 1))
  tf.keras.backend.clear_session()
 # next_values = np.array(next_values).reshape((2, 1))'''

'#n=0\nnext_values = []\ndata_sim = pd.DataFrame(columns=[\'o2sat\', \'sbp\', \'dbp\', \'heartrate\', \'resprate\'])\n#next_df= pd.DataFrame(columns=[\'o2sat\', \'sbp\', \'dbp\', \'heartrate\', \'resprate\'])\nwhile True:\n  #last_row=[]\n  \n  print(data_sim)\n  last_row=df[-1:].to_numpy().astype(\'float32\')\n  last_row=scaler.fit_transform(last_row)\n  for i in range(5):\n    # Predict the next set of values\n    predicted_values_1 = model.predict(last_row)\n    #print(predicted_values_1)\n    # Inverse transform the predicted values using the scaler\n    predicted_values = scaler.inverse_transform(predicted_values_1)\n    print(predicted_values)\n    # Add the predicted values to the list\n    next_values.append(predicted_values)\n    # Update the last row with the predicted values\n    #last_row =  np.vstack([last_row[1:], np.array(next_values[-5:]).reshape((5, last_row.shape[1]))])\n    #last_row =  np.vstack([last_row[1:], np.array(next_values[-5:]).reshape((5, last_row.shape[1]

In [ ]:
n=0
next_values = []
data_sim = pd.DataFrame(columns=['o2sat', 'sbp', 'dbp', 'heartrate', 'resprate'])
#next_df= pd.DataFrame(columns=['o2sat', 'sbp', 'dbp', 'heartrate', 'resprate'])
while True:
  #last_row=[]
  sbp = [rand.randint(sbp_L,sbp_H)]
  dbp = [rand.randint(dbp_L,dbp_H)]
  heartrate = [rand.randint(hr_L,hr_H)]
  o2sat = [rand.randint(o2sat_L,o2sat_H)]
  resprate = [rand.randint(rr_L,rr_H)]
  data={'o2sat':o2sat, 'sbp':sbp, 'dbp':dbp, 'heartrate':heartrate, 'resprate':resprate}
  df2 = pd.DataFrame(data)
  data_sim = pd.concat([data_sim, df2], ignore_index=True)
  print(data_sim)
  last_row=data_sim[-1:].to_numpy().astype('float32')
  last_row=scaler.fit_transform(last_row)
  for i in range(5):
    # Predict the next set of values
    predicted_values_1 = model.predict(last_row)
    #print(predicted_values_1)
    # Inverse transform the predicted values using the scaler
    predicted_values = scaler.inverse_transform(predicted_values_1)
    predicted_values=(predicted_values).astype(int)
    print(predicted_values)
    # Add the predicted values to the list
    next_values.append(predicted_values)
    # Update the last row with the predicted values
    #last_row =  np.vstack([last_row[1:], np.array(next_values[-5:]).reshape((5, last_row.shape[1]))])
    #last_row =  np.vstack([last_row[1:], np.array(next_values[-5:]).reshape((5, last_row.shape[1]))])
    last_row = np.vstack([last_row[1:], predicted_values])
    # Calculate the rolling average of the predicted values for the next 5 rows
  rolling_avg = np.mean(np.array(next_values), axis=0)
  #print("Rolling average of the predicted values:", rolling_avg)
  rolling_df_with_is=pd.DataFrame(rolling_avg,columns=['o2sat', 'sbp', 'dbp', 'heartrate', 'resprate'])
  
  X = scaler.transform(rolling_df_with_is[['o2sat', 'sbp', 'dbp', 'heartrate', 'resprate']])
  y_pred = gb.predict(X)
  rolling_df_with_is['predicted_instability'] = y_pred
  print("Rolling average of the predicted values:" )
  print((rolling_df_with_is).astype(int))
  if y_pred[0] == 1:
    print('======================================')
    print("Instability predicted")
    print('======================================')
  else:
    print('======================================')
  # next_values = np.array(next_values).reshape((5, 1))
  # next_values = np.array(next_values).reshape((5, 1))
  tf.keras.backend.clear_session()
 # next_values = np.array(next_values).reshape((2, 1))

  o2sat sbp dbp heartrate resprate
0   145  75  59       137       22
1/1 [==============================] - 0s 239ms/step
[[145  75  59 137  22]]
1/1 [==============================] - 0s 44ms/step
[[149  70  63 150  18]]
1/1 [==============================] - 0s 73ms/step
[[141  68  64 147  17]]
1/1 [==============================] - 0s 47ms/step
[[142  68  64 147  17]]
1/1 [==============================] - 0s 21ms/step
[[142  68  64 147  17]]
Rolling average of the predicted values:
   o2sat  sbp  dbp  heartrate  resprate  predicted_instability
0    143   69   62        145        18                      0
  o2sat sbp dbp heartrate resprate
0   145  75  59       137       22
1   158  47  42        78       20
1/1 [==============================] - 0s 43ms/step
[[158  47  42  78  20]]
1/1 [==============================] - 0s 36ms/step
[[154  44  46 100  20]]
1/1 [==============================] - 0s 73ms/step
[[150  41  46  93  15]]
1/1 [==============================] - 0s 67ms/st

KeyboardInterrupt: ignored